In [ ]:
def read_data_from_txt(file_path, cams_ratio): 
    # 抽帧，有选择的加载gt文件中的行
    data_dict = {} 
    with open(file_path, 'r') as file:
        for line in file:
            # 从每一行提取tid、fid和bbox
            fid,tid,x1,y1,w,h,_,_,_,_ = map(int, line.strip().split(','))
            data = [tid, x1, y1, w, h]

            if fid % cams_ratio != 0:
                continue
            if fid not in data_dict:
                data_dict[fid] = {'gt':[]} 
            data_dict[fid]['gt'].append(data)

    return data_dict

gt_path = 'datasets/AIC22_Track1_MTMC_Tracking/train/S10/c001/gt/gt.txt'
ratio = 2
gt_dict = read_data_from_txt(gt_path, ratio)
print(len(gt_dict[2]['gt']))

In [1]:
import networkx as nx

# 创建一个空的图
G = nx.Graph()

# 添加轨迹中的节点
G.add_nodes_from([i+1 for i in range(248)])  # 假设有四个物体

# 添加可能的匹配关系
G.add_edges_from([(13, 15), (13, 17), (15, 17), (17, 25), (25, 28), (25, 29), (28, 29), (30, 33),
                  (31, 42), (32, 48), (50, 52), (56, 62), (59, 62), (61, 63), (69, 70), (69, 71),
                  (70, 71), (81, 87), (87, 93), (89, 94), (93, 97), (97, 102), (102, 105), (140, 144),
                  (144, 156), (148, 149), (160, 167), (220, 221), (221, 243), (223, 225), (239, 246)])

matches = nx.max_weight_matching(G)

matching = {match[0]: match[1] for match in matches}
print(matching)


{97: 93, 62: 56, 63: 61, 48: 32, 29: 28, 167: 160, 225: 223, 149: 148, 15: 13, 42: 31, 105: 102, 156: 144, 246: 239, 52: 50, 87: 81, 94: 89, 243: 221, 25: 17, 71: 69, 33: 30}


In [1]:
import networkx as nx

data = [
    (13, 15, 0.055919863534040326),
    (13, 17, 0.08391072678523503),
    (15, 17, 0.04675545819863314),
    (17, 25, 0.0952795234440611),
    (25, 28, 0.026061762853108905),
    (25, 29, 0.07769999145211681),
    (28, 29, 0.06429536213721243),
    (30, 33, 0.011018822541591144),
    (31, 42, 0.0032972466303688996),
    (32, 48, 0.0123805179843548),
    (50, 52, 0.0701107722938773),
    (56, 62, 0.0547255210469495),
    (59, 62, 0.08732068893904366),
    (61, 63, 0.025067058920953844),
    (69, 70, 0.03126330983452186),
    (69, 71, 0.08885931788413104),
    (70, 71, 0.06542229866947713),
    (81, 87, 0.08074798773585701),
    (87, 93, 0.01306705159136512),
    (89, 94, 0.05456472355144193),
    (93, 97, 0.0063138966862459345),
    (97, 102, 0.005156913682248465),
    (102, 105, 0.010204292378161117),
    (140, 144, 0.06941224334162688),
    (144, 156, 0.0019342776181883181),
    (148, 149, 0.039947646171244466),
    (160, 167, 0.07879986959907326),
    (220, 221, 0.08329987716767429),
    (221, 243, 0.08558056966587291),
    (223, 225, 0.031897507880611964),
    (239, 246, 0.07121710780333879)
]

G = nx.Graph()
for edge in data:
    G.add_edge(edge[0], edge[1], weight=-edge[2])  # 负号转换为最小权重匹配

# 使用最大权匹配算法寻找最小权重匹配
matching = nx.max_weight_matching(G, maxcardinality=True)
matching = sorted(matching, key=lambda x: x[0])
sorted_matching = []
for a,b in matching:
    if a > b:
        a, b = b, a
    sorted_matching.append([a,b])
# 计算总的最小权重
total_cost = sum(-G[edge[0]][edge[1]]['weight'] for edge in matching)
print("最小开销的最大匹配：", sorted_matching)
print("总开销：", total_cost)


最小开销的最大匹配： [[13, 15], [17, 25], [28, 29], [30, 33], [31, 42], [32, 48], [50, 52], [56, 62], [61, 63], [69, 70], [81, 87], [89, 94], [93, 97], [102, 105], [144, 156], [148, 149], [160, 167], [220, 221], [223, 225], [239, 246]]
总开销： 0.8822851849597683


In [1]:
# time diff = in time - out time
import itertools
import numpy as np
import os
# 统计每个gt文件中的轨迹id及出现的时间

vdo_rate = 10
cams_path = 'datasets/AIC22_Track1_MTMC_Tracking/train/S03'
timestamp_file = 'datasets/AIC22_Track1_MTMC_Tracking/cam_timestamp/S03.txt'

cam_names = os.listdir(cams_path)
timestamp_dict = {}
with open(timestamp_file, 'r') as time_file:
    for line in time_file:
        cam_name, stamp = line.strip().split(' ')
        timestamp_dict[cam_name] = int(float(stamp) * vdo_rate)
data = {}
for cam in cam_names:
    gt_file = f'{cams_path}/{cam}/gt/gt.txt'
    with open(gt_file, 'r') as file:
        for line in file:
            # 以检测到的最后一帧为时间起点
            last_frame, tid = list(line[:-12].strip().split(','))[:2]
            if (cam not in data.keys()):
                data[cam] = {}
            if tid not in data[cam]:
                # 记录in_time
                data[cam][tid] = {}
                data[cam][tid][0] = int(last_frame) - timestamp_dict[cam]
            # 记录out_time
            data[cam][tid][1] = int(last_frame) - timestamp_dict[cam]

timestamp_differences_dict = {}

# 遍历所有摄像头组合
for (camera1, trajectories1), (camera2, trajectories2) in itertools.combinations(data.items(), 2):
    differences = []

    # 遍历第一个摄像头的轨迹
    for tid, time1 in trajectories1.items():
        # 如果轨迹在第二个摄像头也存在，则计算timestamp差值
        if tid in trajectories2:
            time2 = trajectories2[tid]
            if time1[0] > time2[0]:
                difference = time1[0] - time2[1]
            else:
                difference = time2[0] - time1[1]
            differences.append(difference / vdo_rate)

    key = (camera1, camera2)
    timestamp_differences_dict[key] = differences

for cameras, differences in timestamp_differences_dict.items():
    print(f"Timestamp differences between {cameras[0]} and {cameras[1]}:")
    print(np.mean(differences))

Timestamp differences between c033 and c036:
15.912500000000001
Timestamp differences between c033 and c010:
nan
Timestamp differences between c033 and c014:
nan
Timestamp differences between c033 and c040:
-8.4
Timestamp differences between c036 and c010:
nan
Timestamp differences between c036 and c014:
nan
Timestamp differences between c036 and c040:
5.88
Timestamp differences between c010 and c014:
4.383333333333334
Timestamp differences between c010 and c040:
nan
Timestamp differences between c014 and c040:
nan


/data1/xiayuyang/anaconda3/envs/carla/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/data1/xiayuyang/anaconda3/envs/carla/lib/python3.8/site-packages/numpy/core/_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [ ]:
import motmetrics as mm

acc = mm.MOTAccumulator(auto_id=True)

# data[frame_id]['gt'][2d-list]
# key是帧号

def read_data_from_txt(file_path):
    data_dict = {}  # 最外层字典，key为tid

    with open(file_path, 'r') as file:
        for line in file:
            # 从每一行提取tid、fid和bbox
            fid, tid, x1,y1,w,h,a,b,c,d = map(int, line.strip().split(','))
            bbox = [x1, y1, w, h]

            # 将数据存储到字典中
            if tid not in data_dict:
                data_dict[tid] = {}  # 内层字典，key为fid
            data_dict[tid][fid] = bbox

    return data_dict

gt_dict = read_data_from_txt('datasets/AIC22_Track1_MTMC_Tracking/train/S12/c004/gt/gt.txt')
dt_dict = read_data_from_txt('record/S12_record/my_mot_metrics/c004_gt_test.txt')

data = {}

for tid, v in dt_dict.items():
    for fid, bbox in v.items():
        if fid not in data:
            data[fid] = {'gt':[]}
        data[fid]['gt'].append([tid,bbox[0],bbox[1],bbox[2],bbox[3]])

for tid, v in gt_dict.items():
    for fid, bbox in v.items():
        if fid not in data:
            data[fid] = {}
            data[fid]['gt'] = []
            data[fid]['detections'] = []
        if 'detections' not in data[fid]:
            data[fid]['detections'] = []
        data[fid]['detections'].append([tid,bbox[0],bbox[1],bbox[2],bbox[3]])

for frame, frame_data in data.items():
    gt = frame_data['gt']
    detections = frame_data['detections']
    # 提取gt轨迹id和bbox框
    gt_ids = [item[0] for item in gt]
    gt_bboxes = [item[1:] for item in gt]
    # 提取检测结果id和bbox框
    detection_ids = [item[0] for item in detections]
    detection_bboxes = [item[1:] for item in detections]

    dists = mm.distances.iou_matrix(gt_bboxes, detection_bboxes, max_iou=0.5)

    acc.update(gt_ids, detection_ids, dists)

mh = mm.metrics.create()
summary = mh.compute(acc, metrics=['num_frames', 'idf1', 'idp', 'idr', \
                                     'recall', 'precision', 'num_objects', \
                                     'mostly_tracked', 'partially_tracked', \
                                     'mostly_lost', 'num_false_positives', \
                                     'num_misses', 'num_switches', \
                                     'num_fragmentations', 'mota', 'motp' \
                                    ], \
                      name='acc')
strsummary = mm.io.render_summary(
      summary,
      #formatters={'mota' : '{:.2%}'.format},
      namemap={'idf1': 'IDF1', 'idp': 'IDP', 'idr': 'IDR', 'recall': 'Rcll', \
               'precision': 'Prcn', 'num_objects': 'GT', \
               'mostly_tracked' : 'MT', 'partially_tracked': 'PT', \
               'mostly_lost' : 'ML', 'num_false_positives': 'FP', \
               'num_misses': 'FN', 'num_switches' : 'IDsw', \
               'num_fragmentations' : 'FM', 'mota': 'MOTA', 'motp' : 'MOTP',  \
              }
  )
print(strsummary)


In [1]:
import cv2

def save_frame(video_path, frame_number, save_path):
    # 打开视频文件
    cap = cv2.VideoCapture(video_path)

    # 检查视频是否成功打开
    if not cap.isOpened():
        print("Error: Failed to open video file.")
        return

    # 获取视频的帧率和总帧数
    fps = cap.get(cv2.CAP_PROP_FPS)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    # 检查所需帧是否在视频范围内
    if frame_number < 0 or frame_number >= total_frames:
        print("Error: Invalid frame number.")
        return

    # 设置视频的位置到所需帧
    cap.set(cv2.CAP_PROP_POS_FRAMES, frame_number)

    # 读取所需帧
    ret, frame = cap.read()

    # 检查帧是否成功读取
    if not ret:
        print("Error: Failed to read frame.")
        return

    # 保存帧为图像文件
    cv2.imwrite(save_path, frame)
    print(f"Frame {frame_number} saved as {save_path}")

    # 关闭视频文件
    cap.release()

# 调用函数，传入视频路径、帧数和保存路径
video_path = "datasets/AIC22_Track1_MTMC_Tracking/train/S13/c001/vdo.mp4"  # 替换为你的视频路径
frame_number = 100  # 替换为你想要获取的帧数
save_path = "my_image.jpg"  # 替换为你想要保存的图像路径
save_frame(video_path, frame_number, save_path)


Frame 100 saved as my_image.jpg
